In [ ]:
import pymysql

# MySQL 연결 설정
connection = pymysql.connect(
    host='',      # MySQL 서버 주소 (로컬 호스트라면 'localhost' 또는 127.0.0.1)
    port= ,           # MySQL 사용자 이름
    database="",
    user="",
    password="",
    charset=""
)

try:
    with connection.cursor() as cursor:
        # tb_gis 테이블 생성 SQL 쿼리 작성
        sql = """
        CREATE TABLE IF NOT EXISTS tb_gis (
            id INTEGER(10),          -- 고유 ID
            park_name VARCHAR(40),                     -- 주차장 이름
            address_name VARCHAR(45),                  -- 주차장 주소
            category_group_code VARCHAR(5),           -- 주차장 코드
            park_lat DECIMAL(16, 13),                      -- 주차장 위도 (latitude)
            park_lng DECIMAL(16, 13),                      -- 주차장 경도 (longitude)
            distance INTEGER,                      -- 주차장과 공공기관의 거리
            place_name VARCHAR(70),                    -- 이름
            category_name VARCHAR(70),                 -- 그룹이름
            category_group_name VARCHAR(20),           -- 카테고리 그룹 이름
            road_address_name VARCHAR(45),             -- 주소
            gis_lat DECIMAL(16, 13),                       -- GIS 위도
            gis_lng DECIMAL(16, 13)                       -- GIS 경도
        )
        """
        cursor.execute(sql)  # SQL 쿼리 실행
        print("tb_gis 테이블이 성공적으로 생성되었습니다.")

    # 변경 사항 커밋
    connection.commit()

finally:
    # 연결 종료
    connection.close()


In [ ]:
import pymysql

# 데이터베이스 접속 정보
connection = pymysql.connect(
    host='',
    port=,
    database="",
    user="",
    password="",
    charset=""
)

# 데이터베이스 테이블에서 모든 데이터 삭제 (TRUNCATE 사용)
with connection.cursor() as cursor:
    truncate_query = "TRUNCATE TABLE `tb_gis`"
    cursor.execute(truncate_query)
    connection.commit()

# 연결 종료
connection.close()

print("tb-gis 테이블의 모든 데이터를 성공적으로 삭제했습니다.")

In [ ]:
import pandas as pd
import pymysql

# CSV 파일 불러오기
csv_file_path = 'C:/Users/USER/Desktop/Final/category_output_data_241011.csv'
data = pd.read_csv(csv_file_path)

# NaN 값을 빈 문자열로 대체
data.fillna('', inplace=True)

# 데이터베이스 접속 정보
connection = pymysql.connect(
    host='',
    port=,
    database="",
    user="",
    password="",
    charset=""
)

# 검색 포인트에서 park_lng와 park_lat 추출
data['park_lng'] = data['search_point'].str.extract(r'^\s*\(\s*([-\d.]+),')[0].astype(float)
data['park_lat'] = data['search_point'].str.extract(r',\s*([-\d.]+)\s*\)')[0].astype(float)

# 삽입할 데이터 준비
insert_data = data[['id', 'park_name', 'address_name', 'category_group_code', 'park_lat', 'park_lng', 'distance', 'place_name', 'category_name', 'category_group_name', 'road_address_name', 'gis_lat', 'gis_lng']]

# SQL INSERT 쿼리
insert_query = """
    INSERT INTO `tb_gis` (id, park_name, address_name, category_group_code, park_lat, park_lng, distance, place_name, category_name, category_group_name, road_address_name, gis_lat, gis_lng) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# 데이터 삽입
with connection.cursor() as cursor:
    for index, row in insert_data.iterrows():
        cursor.execute(insert_query, tuple(row))
        if index % 1000 == 0:  # 1000개마다 커밋
            connection.commit()
    connection.commit()  # 마지막 커밋

# 연결 종료
connection.close()

print("모든 데이터를 성공적으로 삽입했습니다.")


In [1]:
import pymysql

# 데이터베이스 접속 정보
connection = pymysql.connect(
    host='',
    port=,
    database="",
    user="",
    password="",
    charset=""
)

# 검색을 위한 함수 (API 검색 기능)
def search_api(keyword):
    # API 호출을 구현해야 하는 부분 (가상의 API 예시)
    # 실제 API로부터 keyword와 일치하는 검색 결과를 반환
    search_results = ["서울특별시 강남구", "부산광역시 해운대구"]  # 예시 결과
    return search_results

# 테이블에서 모든 address 가져오기
with connection.cursor() as cursor:
    select_query = "SELECT id, address_name FROM `tb_gis`"
    cursor.execute(select_query)
    rows = cursor.fetchall()

# 검색어로 결과 필터링
search_keyword = "강남"  # 검색할 키워드
matched_results = search_api(search_keyword)
filtered_data = [row for row in rows if any(result in row[1] for result in matched_results)]

# 가져온 데이터에서 검색 결과를 제외한 나머지 데이터 삭제 (파이썬 내에서)
filtered_ids = [row[0] for row in filtered_data]
filtered_addresses = [row[1] for row in filtered_data]

# 출력해서 남은 데이터 확인
print("검색 결과에 해당하는 데이터:")
for id, address in zip(filtered_ids, filtered_addresses):
    print(f"ID: {id}, Address: {address}")


# 연결 종료
connection.close()


검색 결과에 해당하는 데이터:


In [9]:
import pymysql

# 데이터베이스 접속 정보
connection = pymysql.connect(
    host='',
    port=,
    database="",
    user="",
    password="",
    charset=""
)

# 검색을 위한 함수 (API 검색 기능)
def search_api(keyword):
    # 실제 API 구현 부분 (예시 결과 반환)
    search_results = ["서울특별시 강남구", "부산광역시 해운대구"]
    return search_results

# 테이블에서 모든 address_name 가져오기
with connection.cursor() as cursor:
    select_query = "SELECT address_name FROM `tb_gis`"
    cursor.execute(select_query)
    rows = cursor.fetchall()

# 검색 키워드로 필터링
search_keyword = "신길동 103-9"  # 검색할 키워드

# 검색 키워드가 포함된 주소만 남기기
filtered_data = [row for row in rows if search_keyword in row[1]]

# 필터링된 데이터에서 ID와 주소 분리
filtered_ids = [row[0] for row in filtered_data]
filtered_addresses = [row[1] for row in filtered_data]

# 남은 데이터 출력
print("검색 결과에 해당하는 데이터:")
if filtered_data:
    for id, address in zip(filtered_ids, filtered_addresses):
        print(f"ID: {id}, Address: {address}")
else:
    print("검색 결과가 없습니다.")

# 연결 종료
connection.close()


IndexError: tuple index out of range

In [ ]:
import pymysql

# 데이터베이스 접속 정보
connection = pymysql.connect(
    host='',
    port=,
    user='',
    password='',
    database='',  # 변경된 DB 이름
    charset=''
)

# 검색을 위한 함수 (API 검색 기능)
def search_api(keyword):
    # 실제 API 구현 부분 (예시 결과 반환)
    search_results = ["서울특별시 강남구", "부산광역시 해운대구"]
    return search_results

# 테이블에서 모든 address_name 가져오기
with connection.cursor() as cursor:
    select_query = "SELECT address_name FROM `tb_gis`"  # address_name만 선택
    cursor.execute(select_query)
    rows = cursor.fetchall()

# 검색 키워드로 필터링
search_keyword = "구로구"  # 검색할 키워드

# 검색 키워드가 포함된 주소만 남기기
filtered_addresses = [row[0] for row in rows if search_keyword in row[0]]

# 남은 데이터 출력
print("검색 결과에 해당하는 데이터:")
if filtered_addresses:
    for address in filtered_addresses:
        print(address)
else:
    print("검색 결과가 없습니다.")

# 연결 종료
connection.close()


In [ ]:
import pymysql

# 데이터베이스 접속 정보
connection = pymysql.connect(
    host='',
    port=,
    user='',
    password='',
    database='',  # 변경된 DB 이름
    charset=''
)

# # 검색을 위한 함수 (API 검색 기능)
# def search_api(keyword):
#     # 실제 API 구현 부분 (예시 결과 반환)
#     search_results = ["서울특별시 강남구", "부산광역시 해운대구"]
#     return search_results

# 테이블에서 모든 address_name 가져오기
with connection.cursor() as cursor:
    select_query = "SELECT address_name FROM `tb_gis`"
    cursor.execute(select_query)
    rows = cursor.fetchall()

# 검색 키워드로 필터링
search_keyword = "영등포"  # 검색할 키워드

# '서울 영등포구'로 시작하는 주소만 남기기
filtered_addresses = [
    row[0] for row in rows 
    if search_keyword in row[0] and row[0].startswith("서울 영등포구")
]

# 남은 데이터 출력
print("검색 결과에 해당하는 데이터:")
if filtered_addresses:
    for address in filtered_addresses:
        print(address)
else:
    print("검색 결과가 없습니다.")

# 연결 종료
connection.close()


In [33]:
import pymysql

# 데이터베이스 접속 정보
connection = pymysql.connect(
  
)

try:
    with connection.cursor() as cursor:
        # TRIM() 함수 사용으로 공백 제거 후 주소 매칭
        select_query = """
            SELECT 
                a.address, a.lat, a.lng
            FROM 
                tb_address a
            INNER JOIN 
                tb_gis g 
            ON 
                TRIM(a.address) = TRIM(g.address_name)
            WHERE 
                a.address LIKE '%영등포%'
        """
        cursor.execute(select_query)
        rows = cursor.fetchall()

        # 결과 출력
        print("검색 결과에 해당하는 데이터:")
        if rows:
            for row in rows:
                print(f"{row[0]} (Lat: {row[1]}, Lng: {row[2]})")
        else:
            print("검색 결과가 없습니다.")

finally:
    # 연결 종료
    connection.close()


검색 결과에 해당하는 데이터:
검색 결과가 없습니다.


In [ ]:
try:
    with connection.cursor() as cursor:
        # 영등포 포함된 주소만 각각 조회
        query_address = "SELECT address FROM tb_address WHERE address LIKE '%영등포%'"
        query_gis = "SELECT address_name FROM tb_gis WHERE address_name LIKE '%영등포%'"

        cursor.execute(query_address)
        address_rows = cursor.fetchall()

        cursor.execute(query_gis)
        gis_rows = cursor.fetchall()

        print("비교 결과:")
        for addr in address_rows:
            if addr not in gis_rows:
                print(f"매칭되지 않는 주소: {addr[0]}")

finally:
    connection.close()


## tb_gis의 address_name, road_address_name과 tb_address의 address중의 일치하는 것을 찾는것

In [44]:
import pymysql

# 데이터베이스 접속 정보
connection = pymysql.connect(

)

try:
    with connection.cursor() as cursor:
        # tb_address에서 영등포가 포함된 주소 가져오기
        query_address = """
            SELECT TRIM(address) AS address
            FROM tb_address
            WHERE address LIKE '%영등포%'
        """
        cursor.execute(query_address)
        address_rows = cursor.fetchall()

        # tb_gis에서 영등포가 포함된 address_name 가져오기
        query_gis = """
            SELECT TRIM(address_name) AS address_name
            FROM tb_gis
            WHERE address_name LIKE '%영등포%'
        """
        cursor.execute(query_gis)
        gis_rows = cursor.fetchall()

        # 주소 리스트 만들기
        address_list = [row[0] for row in address_rows]
        gis_list = [row[0] for row in gis_rows]

        # 주소 일치 확인
        matching_addresses = set(address_list) & set(gis_list)  # 교집합

        # 결과 출력
        if matching_addresses:
            print("일치하는 주소:")
            for match in matching_addresses:
                print(match)
        else:
            print("일치하는 주소가 없습니다.")

finally:
    # 연결 종료
    connection.close()


일치하는 주소가 없습니다.


## tb_address에서 '영등포'가 포함된 주소, 건물 이름, 위도, 경도 가져오기

In [ ]:
import pymysql

# 데이터베이스 접속 정보


try:
    with connection.cursor() as cursor:
        # tb_address에서 '영등포'가 포함된 주소, 건물 이름, 위도, 경도 가져오기
        select_query = """
            SELECT 
                TRIM(address) AS address, 
                TRIM(building_name) AS building_name, 
                lat, 
                lng 
            FROM 
                tb_address 
            WHERE 
                address LIKE '%영등포%'
        """
        cursor.execute(select_query)
        rows = cursor.fetchall()

        # 결과 출력
        print("검색 결과에 해당하는 데이터:")
        if rows:
            for row in rows:
                address, building_name, lat, lng = row
                # 건물명이 None일 경우 빈 문자열로 설정
                building_name_display = building_name if building_name else "없음"
                print(f"{address}, {building_name_display}, {lat},{lng}")
        else:
            print("검색 결과가 없습니다.")

finally:
    # 연결 종료
    connection.close()


In [ ]:
import pymysql

# 데이터베이스 접속 정보
connection = pymysql.connect(
    
)
# 검색 키워드
search_keyword = "강남구"

try:
    with connection.cursor() as cursor:
        # tb_address에서 검색어가 포함된 주소, 위도, 경도 가져오기
        select_query = """
            SELECT 
                TRIM(address) AS address, 
                lat, 
                lng 
            FROM 
                tb_address 
            WHERE 
                address LIKE %s
        """
        # LIKE 쿼리를 위해 검색어에 와일드카드를 추가
        cursor.execute(select_query, ('%' + search_keyword + '%',))
        rows = cursor.fetchall()

        # 결과 출력
        print("검색 결과에 해당하는 데이터:")
        if rows:
            for row in rows:
                address, lat, lng = row
                print(f"{address}, {lat}, {lng}")
        else:
            print("검색 결과가 없습니다.")

finally:
    # 연결 종료
    connection.close()


## 영등포 이외는 검색 X

In [16]:
import pymysql

# 데이터베이스 접속 정보
connection = pymysql.connect(

)

# 검색 키워드
search_keyword = "강북구"

try:
    with connection.cursor() as cursor:
        # tb_address에서 '영등포'가 포함된 주소, 위도, 경도 가져오기
        select_query = """
            SELECT 
                TRIM(address) AS address, 
                lat, 
                lng 
            FROM 
                tb_address 
            WHERE 
                address LIKE %s
        """
        # SQL 쿼리에서 '영등포'가 포함된 주소만 가져오도록 설정
        cursor.execute(select_query, ('%' + search_keyword + '%',))
        rows = cursor.fetchall()

        # '서울 영등포구'와 같은 영등포 지역만 필터링
        filtered_rows = [
            row for row in rows if "영등포" in row[0]
        ]

        # 결과 출력
        print("검색 결과에 해당하는 데이터:")
        if filtered_rows:
            for address, lat, lng in filtered_rows:
                print(f"{address}, {lat}, {lng}")
        else:
            print("검색 결과가 없습니다.")

finally:
    # 연결 종료
    connection.close()


검색 결과에 해당하는 데이터:
검색 결과가 없습니다.
